In [ ]:
import random
import warnings
import numpy as np
import matplotlib.pyplot as plt
import dataLoader as dl
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#from Inner_Speech_Dataset.Plotting.ERPs import 
from Inner_Speech_Dataset.Python_Processing.Data_extractions import  Extract_data_from_subject
from Inner_Speech_Dataset.Python_Processing.Data_processing import  Select_time_window, Transform_for_classificator, Split_trial_in_time
from Inner_Speech_Dataset.Python_Processing.Data_processing import  Calculate_power_windowed
from Inner_Speech_Dataset.Python_Processing.Utilitys import picks_from_channels
from Inner_Speech_Dataset.Python_Processing.Data_processing import Average_in_frec
#Frequencies

from scipy.fft import rfft, ifft, fftshift, fftfreq


#Separate into equal 5 buckets
def sepFreqIndexBuckets(freqs2, nr_of_buckets = 5): 
     
    bucket_size_amp = np.sum(freqs2)/nr_of_buckets
    #print(bucket_size_amp)
    
    buckets = np.zeros([nr_of_buckets, 2])
    bucket = []
    cur_buck_size = 0
    
    b = 0
    c = 0
    for index, freqs in enumerate(freqs2,0):
        cur_buck_size += freqs
        bucket.append(index)
        if cur_buck_size > bucket_size_amp:
            buckets[b] = [0 + c , c + len(bucket)]
            b += 1
            c += len(bucket)
            #print(len(bucket))
            bucket = []
            cur_buck_size = 0
            
    
    buckets[b] = [0 + c , c + len(bucket)]
    #print(len(bucket)) 
       
    return buckets  




def createFreqBuckets(data, nr_of_buckets = 5):


    nr_of_buckets = 5
    buckets = np.zeros([nr_of_buckets, 2])
    for trial in data:
        for channel in trial:
            buckets += sepFreqIndexBuckets(abs(rfft(channel))[:(channel.shape[0]//2)]*1000, nr_of_buckets)
            
    buckets = buckets/(data.shape[0]*data.shape[1])
    
    
    return np.int32(buckets)


def data_into_freq_buckets(data, nr_of_buckets, buckets):

    freqAmps = np.zeros([data.shape[0], data.shape[1], nr_of_buckets])
    for tr_nr, trial in enumerate(data):
        for ch_nr, channel in enumerate(trial):
            for b in range(nr_of_buckets):
                ff_c = abs(rfft(channel))*1000
                freqAmps[tr_nr, ch_nr, b] = np.sum(ff_c[int(buckets[b, 0]):int(buckets[b,1])])
    return freqAmps

#Channel name array

def arrToDict(arr):
    dict = {}
    for row in arr:
        dict[row[0]] = row[1]
    
    return dict

def get_channelNames():
    ch_names = np.array(dl.get_channelnames())
    nr = np.arange(ch_names.shape[0])
    ch_names = np.array([ch_names, nr]).T
    ch_names = arrToDict(ch_names)
    return ch_names

def only_spec_channel_data(data , picks):
    
    channel_names_string = picks_from_channels(picks)
    ch_names = get_channelNames()
    channel_nr = []
    for name in  channel_names_string:
        channel_nr.append(int(ch_names[name]))
        #print(ch_names[name])

    channel_nr = np.array(channel_nr)
    
    #print(channel_nr)
    #data = np.swapaxes(data, 0, 1)
    #labels = np.swapaxes(labels, 0, 1)
    #for channelnrs in channels:
    data2 = np.delete(data, np.delete(np.arange(128), channel_nr) , axis=1)
    return data2


def get_power_array(split_data , samplingRate, trialSplit = 1, t_min = 0, t_max = 0.99):

    #trialSplit = 16
    sR = samplingRate #samplingRate = 32
    data_power = np.zeros([split_data.shape[0], split_data.shape[1], trialSplit, 2])
    for t, trial in enumerate(split_data,0):
        for c, channel in enumerate(trial,0):
            for x in range(trialSplit):
                data_power[t, c, x, : ] = Calculate_power_windowed(channel, fc=sR, window_len=1/8, window_step=1/8, t_min=t_min*(1/trialSplit), t_max=t_max*(1/trialSplit))


    #m_power , std_power
    #print(data_power.shape)
    return data_power
    


#Loading the data and labels from EEG and EXG

# data1, labels1 = dl.load_data(datatype="EEG", subject_nr=1, verbose=True,sampling_rate=sampling_rate) 
# data2, labels2 = dl.load_data(datatype="EEG", subject_nr=2 ,verbose=True,sampling_rate=sampling_rate )
# data4 , labels4 = dl.load_data(datatype="EEG", subject_nr=4, verbose=True,sampling_rate=sampling_rate) 

# dataX, labelsX = dl.load_data(datatype="EXG", verbose=False) 
# #datab, labelsb = dl.load_data(datatype="baseline", verbose=False, sampling_rate=32) 
# #dl.load_data(datatype2=2) #4.5 is max

####

#data = np.concatenate([data1, data2, data4], axis = 0)
#labels1d = np.concatenate([labels1, labels2, labels4], axis = 0)

####

#data = data1
#labels1d = labels1

sampling_rate = 128
data, labels = dl.load_multiple_datasets(nr_of_datasets=8, sampling_rate=sampling_rate, t_min=2, t_max=3)
ch_names = get_channelNames()

# print(data.shape)
# dataCL = only_spec_channel_data(data, "CL")
# dataCZ = only_spec_channel_data(data, "CZ")
# dataPZ = only_spec_channel_data(data, "PZ")
# dataOPZ = only_spec_channel_data(data, "OPZ")

# def avg_channels(data):
#     avg_data = np.mean(data, axis=1)
#     print(avg_data.shape)
#     return np.reshape(avg_data, [avg_data.shape[0], 1 , avg_data.shape[1]])

# data = np.concatenate([avg_channels(dataCL), avg_channels(dataCZ),
#                 avg_channels(dataPZ), avg_channels(dataOPZ),], axis=1)
# print(data.shape)

#Not work when time is not 4.5 right now kinda
print(data.shape)
data_p =  get_power_array(data, sampling_rate, trialSplit=1).squeeze()
print(data_p.shape)
#print(data_p[:,:,1])

#Getting Freq Data 
nr_of_buckets = 5
buckets = createFreqBuckets(data, nr_of_buckets)
print("buckets")
print(buckets)
data_f = data_into_freq_buckets(data, nr_of_buckets, buckets)
print(data_f.shape)


## Normalize data


print(data_f.shape)
print(data_p.shape)
print(labels.shape)
print(data_p[2,5])
data = np.concatenate([data_f, data_p, data], axis =2 )
print(data.shape)
data = keras.utils.normalize(data, axis=1, order=2)
print(data.shape)
#print(data[2,5])
#print(labels[2])
print(labels[:].shape)
# def get_local_frequency(split_data , samplingRate, trialSplit = 2,):

#     #trialSplit = 16
#     sR = samplingRate #samplingRate = 32
#     data_power = np.zeros([split_data.shape[0], split_data.shape[1], trialSplit, 2])
#     for t, trial in enumerate(split_data,0):
#         for c, channel in enumerate(trial,0):
#             for x in range(trialSplit):
#                 data_power[t, c, x, : ] = Calculate_power_windowed(channel, fc=sR, window_len=1*4/trialSplit, window_step=1*4/trialSplit, t_min=0, t_max=4/trialSplit * x + 4/trialSplit)
    

#     #m_power , std_power
#     #print(data_power.shape)
#     return data_power
    
#Splitting into training and test data
#print(labels)


order = np.arange(labels.shape[0])
np.random.shuffle(order)

temp_data = np.zeros(data.shape)
temp_labels = np.zeros(labels.shape)

for x in range(labels.shape[0]):
    i = order[x]
    
    temp_data[x] = data[i]
    temp_labels[x] = labels[i]

data = temp_data
labels = temp_labels

data_train, data_test = np.split(data, indices_or_sections=[int(labels.shape[0]*0.75)],axis=0)
labels_train, labels_test = np.split(labels, indices_or_sections=[int(labels.shape[0]*0.75)],axis=0)
print(labels_train.shape)
print(data_train.shape)
print(data_test.shape)



#Convolution part

# Do the convs like I wrote in the Note
# First need to find out 8 best channels
#  Utilities can help average similar channels
# Read through article and how they did it better!
# convlayer = tf.keras.layers.Conv2D(4,
#  kernel_size=[2,1], input_shape= (data_train.shape[0], data_train.shape[1] , data_train.shape[2], 1,), 
#  padding="valid", strides=[round(data_train.shape[1]),1],
#  activation="relu")


# print(data_train.shape)
# print(data_test.shape)

# #data_train = np.swapaxes(data_train, 2, 1)
# #data_test = np.swapaxes(data_test, 2, 1)
# print(data_train.shape)
# print(data_test.shape)
# data_train = np.expand_dims(data_train, axis=0)
# data_test = np.expand_dims(data_test, axis=0)
# data_train = np.moveaxis(data_train, 0, -1)
# data_test = np.moveaxis(data_test, 0, -1)

# conv_data_train = convlayer(data_train)
# conv_data_test = convlayer(data_test)

# data_train = conv_data_train
# data_test = conv_data_test

# #data_train = np.swapaxes(data_train, 2, 1)
# #data_test = np.swapaxes(data_test, 2, 1)


# data_train = np.squeeze(np.moveaxis(data_train, -1, 0))
# data_test = np.squeeze(np.moveaxis(data_test, -1, 0))
# print(data_train.shape)
# print(data_test.shape)


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.keras.backend.clear_session()





eeg_model = tf.keras.Sequential([
    
    #layers.Conv2D(500,
    #kernel_size=[1,8], input_shape= (1, data_train.shape[1], data_train.shape[2], 1,), 
    #padding="valid", strides=[1,4],
    #activation="relu"),
    # layers.Conv2D(200,
    # kernel_size=[1,32], 
    # padding="valid", strides=1,
    # activation="relu"),
    # layers.Conv2D(200,
    # kernel_size=[1,32], 
    # padding="valid", strides=1,
    # activation="relu"),
    # layers.Conv2D(200,
    # kernel_size=[1,32], 
    # padding="valid", strides=4,
    # activation="relu"),
    layers.LocallyConnected1D(128, input_shape = (data_train.shape[1],data_train.shape[2]), 
     kernel_size=50, 
     padding="valid", strides=1,
     activation="relu"),
    layers.Flatten(input_shape = (data_train.shape[1],data_train.shape[2])),
    #layers.Dense(units=28*3, activation="relu"),
    #layers.Dropout(0.3),
    #layers.Dense(units=28*28, activation="relu"),
    #layers.Dropout(0.3),
    layers.Dense(units=28*28, activation="relu"),
    #layers.Dense(units=20, activation="relu"),
    #layers.Dense(units=5, activation="relu"),
    #layers.Dense(units=1, activation="relu"),
    #layers.Flatten(),
    layers.Dropout(0.1),
    layers.Dense(units=28*28, activation="relu"),
    layers.Dropout(0.1),
    layers.Dense(units=28, activation="relu"),
    layers.Dropout(0.1),
    layers.Dense(units=2, activation="softmax")


])
eeg_model.build()
eeg_model.summary()


for trialNr, trial in enumerate(data_train[43:47],1):
    for channel in trial[3:4]:
        plt.figure()
        plt.plot(channel)
        plt.title("EEG {}".format(labels_train[trialNr]))

print(labels[2])
print(labels[:].shape)
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
callback2 = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)
#tensorboard_callback = tf.keras.callbacks.TensorBoard(histogram_freq=2)

eeg_model.compile(optimizer='adam',
              loss="binary_crossentropy",
              metrics=['accuracy'])

data_train_send = np.reshape(data_train, [data_train.shape[0], 1 , data_train.shape[1], data_train.shape[2], 1])
#data_train_send = np.reshape(data_train, [data_train.shape[0], 1 , data_train.shape[1], data_train.shape[2], 1])

data_test_send = np.reshape(data_test, [data_test.shape[0], 1 , data_test.shape[1], data_test.shape[2], 1])
#data_test_send = np.reshape(data_test, [data_test.shape[0], 1 , data_test.shape[1], data_test.shape[2], 1])

outputs = eeg_model.fit(data_train, labels_train, callbacks=[callback, callback2], epochs=20) #

print("Results")
eeg_model.evaluate(data_test, labels_test)
result = eeg_model.predict(data_test)

result2 = []
print(result)
for res in result:
    x = np.maximum(res[0], res[1])
    result2.append(int(np.where(res == x)[0]))

result2 = np.array(result2)
print(result2)



for trialNr, trial in enumerate(data[44:46],44):
    for channel in trial[5:6]:
        plt.figure()
        plt.plot(channel)
        plt.title("EEG {}".format(labels[trialNr]))

      
